In [3]:
# dependicies
# import tensorflow
import numpy as np
from keras.applications import vgg16
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array

# helper class
import image_processing as img

# image display
from IPython.display import Image

In [4]:
# get tensor representations of our images
base_image = K.variable(img.preprocess_image('./base_image.jpg'))
style_reference_image = K.variable(img.preprocess_image('./style_image.jpg'))
combination_image = K.placeholder((1, 400, 711, 3))

Image('./style_image.jpg')

AttributeError: 'module' object has no attribute 'preprocess_image'